1. Setup and Data Preprocessing

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K

# Load dataset
data = pd.read_csv('dataset/Almond.csv')

# Display initial data
print("Initial Data:")
display(data.head())

# Handle missing values
print("\nMissing Values Before Imputation:")
print(data.isnull().sum())

# Separate numeric and non-numeric columns
numeric_cols = data.select_dtypes(include=['number']).columns
non_numeric_cols = data.select_dtypes(exclude=['number']).columns

# Fill missing values in numeric columns with the mean
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Fill missing values in non-numeric columns with the mode (most frequent value)
data[non_numeric_cols] = data[non_numeric_cols].apply(lambda x: x.fillna(x.mode()[0]))

print("\nMissing Values After Imputation:")
print(data.isnull().sum())

# Feature Engineering
if 'Length' in data.columns and 'Width' in data.columns:
    data['Aspect_Ratio'] = data['Length'] / data['Width']
else:
    print("\nColumns 'Length' and 'Width' not found for aspect ratio calculation.")

# Encode the labels
label_encoder = LabelEncoder()
data['Type'] = label_encoder.fit_transform(data['Type'])

# Split features and target
X = data.drop('Type', axis=1).values
y = data['Type'].values

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=52
)

# Check the number of classes
n_classes = len(np.unique(y))
print(f"\nNumber of classes: {n_classes}")

Initial Data:


,Unnamed: 0,Length (major axis),Width (minor axis),Thickness (depth),Area,Perimeter,Roundness,Solidity,Compactness,Aspect Ratio,Eccentricity,Extent,Convex hull(convex area),Type
0,0,NaN,227.940628,127.759132,22619.0,643.813269,NaN,0.973384,1.458265,NaN,NaN,0.681193,23237.5,MAMRA
1,1,NaN,234.188126,128.199509,23038.0,680.984841,NaN,0.957304,1.601844,NaN,NaN,0.656353,24065.5,MAMRA
2,2,NaN,229.418610,125.796547,22386.5,646.943212,NaN,0.967270,1.487772,NaN,NaN,0.683620,23144.0,MAMRA
3,3,NaN,232.763153,125.918808,22578.5,661.227483,NaN,0.965512,1.540979,NaN,NaN,0.685360,23385.0,MAMRA
4,4,NaN,230.150742,107.253448,19068.0,624.842706,NaN,0.951450,1.629395,NaN,NaN,0.714800,20041.0,MAMRA



Missing Values Before Imputation:
Unnamed: 0                     0
Length (major axis)          857
Width (minor axis)           942
Thickness (depth)           1004
Area                           0
Perimeter                      0
Roundness                    857
Solidity                       0
Compactness                    0
Aspect Ratio                1799
Eccentricity                1799
Extent                         0
Convex hull(convex area)       0
Type                           0
dtype: int64

Missing Values After Imputation:
Unnamed: 0                  0
Length (major axis)         0
Width (minor axis)          0
Thickness (depth)           0
Area                        0
Perimeter                   0
Roundness                   0
Solidity                    0
Compactness                 0
Aspect Ratio                0
Eccentricity                0
Extent                      0
Convex hull(convex area)    0
Type                        0
dtype: int64

Columns 'Length' and '

2. Defining Neural Network Model

In [2]:
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))

    # Choose the optimizer based on the input string
    if optimizer == 'adam':
        model.compile(
            optimizer=Adam(),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
    elif optimizer == 'rprop':
        model.compile(
            optimizer=RMSprop(),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
    elif optimizer == 'hybrid':
        model.compile(
            optimizer=HybridOptimizer(),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
    else:
        raise ValueError(f"Unknown optimizer: {optimizer}")

    return model

3. Defining Hybrid Optimizer

In [3]:
def hybrid_optimizer(model, X_train, y_train, X_test, y_test, epochs=50):
    # Optimizers
    adam_opt = Adam(learning_rate=0.0001)
    rprop_opt = RMSprop(learning_rate=0.0001)

    # Compile model with Adam
    model.compile(
        optimizer=adam_opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    history_adam = model.fit(
        X_train, y_train, validation_data=(X_test, y_test),
        epochs=epochs, batch_size=16, verbose=0
    )
    adam_weights = model.get_weights()
    adam_loss = history_adam.history['loss'][-1]

    # Compile model with RProp
    model.compile(
        optimizer=rprop_opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    history_rprop = model.fit(
        X_train, y_train, validation_data=(X_test, y_test),
        epochs=epochs, batch_size=16, verbose=0
    )
    rprop_weights = model.get_weights()
    rprop_loss = history_rprop.history['loss'][-1]

    # Average the weights
    hybrid_weights = [
        (0.5 * a + 0.5 * r) for a, r in zip(adam_weights, rprop_weights)
    ]
    model.set_weights(hybrid_weights)

    return model, history_adam, history_rprop

4. Training Loop and Correlation Calculation

In [4]:
def run_experiments(n_runs, epochs):
    adam_accuracies = []
    rprop_accuracies = []
    hybrid_accuracies = []
    adam_losses = []
    rprop_losses = []
    hybrid_losses = []
    correlations = []

    best_hybrid_acc = 0
    best_run_index = -1

    for run in range(n_runs):
        # Create a new model instance for each run
        model = create_model()
        
        # Adding EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train with hybrid optimizer
        model, history_adam, history_rprop = hybrid_optimizer(model, X_train, y_train, X_test, y_test, epochs)

        # Collect accuracies
        adam_acc = history_adam.history['accuracy'][-1]
        rprop_acc = history_rprop.history['accuracy'][-1]
        hybrid_acc = model.evaluate(X_test, y_test, verbose=0)[1]
        
        # Collect losses
        adam_loss = history_adam.history['loss']
        rprop_loss = history_rprop.history['loss']
        hybrid_loss = model.evaluate(X_test, y_test, verbose=0)[0]  # Loss from the evaluation of the final model

        adam_accuracies.append(adam_acc)
        rprop_accuracies.append(rprop_acc)
        hybrid_accuracies.append(hybrid_acc)
        
        adam_losses.append(adam_loss)
        rprop_losses.append(rprop_loss)
        hybrid_losses.append(hybrid_loss)

        # Print the accuracies for the current run
        print(f"Run {run + 1}: Adam Accuracy = {adam_acc:.4f}, RProp Accuracy = {rprop_acc:.4f}, Hybrid Accuracy = {hybrid_acc:.4f}")

        # Update the best run based on hybrid accuracy
        if hybrid_acc > best_hybrid_acc:
            best_hybrid_acc = hybrid_acc
            best_run_index = run + 1

    # Printing the best run
    print(f"\nBest Run: {best_run_index} with Hybrid Accuracy = {best_hybrid_acc:.4f}")

    # Calculate and print average and standard deviation for each optimizer
    print("\nSummary of Results:")
    print(f"Adam - Average Accuracy: {np.mean(adam_accuracies):.4f}, Std Dev: {np.std(adam_accuracies):.4f}")
    print(f"RProp - Average Accuracy: {np.mean(rprop_accuracies):.4f}, Std Dev: {np.std(rprop_accuracies):.4f}")
    print(f"Hybrid - Average Accuracy: {np.mean(hybrid_accuracies):.4f}, Std Dev: {np.std(hybrid_accuracies):.4f}\n")

    return adam_accuracies, rprop_accuracies, hybrid_accuracies, adam_losses, rprop_losses, hybrid_losses, correlations


5. Defining Grid Search for Hyperparameter Optimization

In [ ]:
# Cross-validation setup
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)

# Collect metrics across folds
adam_acc_list = []
rprop_acc_list = []
hybrid_acc_list = []
adam_loss_list = []
rprop_loss_list = []
hybrid_loss_list = []
final_y_tests = []  # To store the actual labels for the last fold
final_y_preds = []  # To store predictions for the last fold

# Create a new model instance for each fold
for fold_idx, (train_index, test_index) in enumerate(skf.split(X_scaled, y)):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Run the experiments for the current fold
    adam_accuracies, rprop_accuracies, hybrid_accuracies, adam_loss, rprop_loss, hybrid_loss, correlations = run_experiments(n_runs=10, epochs=50)

    # Collect accuracies for the current fold
    adam_acc_list.append(adam_accuracies)
    rprop_acc_list.append(rprop_accuracies)
    hybrid_acc_list.append(hybrid_accuracies)

    # Collect loss history for the current fold
    adam_loss_list.append(adam_loss)
    rprop_loss_list.append(rprop_loss)
    hybrid_loss_list.append(hybrid_loss)

    # Store the actual labels for the last fold
    final_y_tests.append(y_test)

# Averaging results across folds
adam_avg_acc = np.mean(adam_acc_list, axis=0)
rprop_avg_acc = np.mean(rprop_acc_list, axis=0)
hybrid_avg_acc = np.mean(hybrid_acc_list, axis=0)

# Plotting accuracies across folds
methods = ['Adam', 'RProp', 'Hybrid']
accuracies = [adam_avg_acc, rprop_avg_acc, hybrid_avg_acc]

plt.figure(figsize=(10, 6))
plt.boxplot(accuracies, labels=methods)
plt.title('Accuracy Comparison Across Different Optimizers')
plt.ylabel('Accuracy')
plt.show()

# Plotting Loss Over Epochs for Adam, RProp, and Hybrid
plt.figure(figsize=(10, 6))
for fold_idx, (adam_loss, rprop_loss, hybrid_loss) in enumerate(zip(adam_loss_list, rprop_loss_list, hybrid_loss_list)):
    plt.plot(adam_loss, label=f'Adam Loss (Fold {fold_idx + 1})', linestyle='dotted')
    plt.plot(rprop_loss, label=f'RProp Loss (Fold {fold_idx + 1})', linestyle='solid')
    plt.plot(hybrid_loss, label=f'Hybrid Loss (Fold {fold_idx + 1})', linestyle='dashed')
plt.title('Loss Over Epochs for Adam, RProp, and Hybrid (Across Folds)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot confusion matrices for each optimizer in the last fold
final_y_test = final_y_tests[-1]
final_y_pred_classes = final_y_preds[-1]

# Confusion Matrix for Hybrid
conf_matrix_hybrid = confusion_matrix(final_y_test, final_y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_hybrid, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix for Last Fold (Hybrid Optimizer)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Print classification report for the last fold
print(classification_report(final_y_test, final_y_pred_classes))

c:\Users\edwin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Run 1: Adam Accuracy = 0.7908, RProp Accuracy = 0.8586, Hybrid Accuracy = 0.8235
Run 2: Adam Accuracy = 0.7846, RProp Accuracy = 0.8359, Hybrid Accuracy = 0.8378
Run 3: Adam Accuracy = 0.7578, RProp Accuracy = 0.8595, Hybrid Accuracy = 0.8414
Run 4: Adam Accuracy = 0.7632, RProp Accuracy = 0.8501, Hybrid Accuracy = 0.8307
Run 5: Adam Accuracy = 0.7703, RProp Accuracy = 0.8524, Hybrid Accuracy = 0.8592
Run 6: Adam Accuracy = 0.7868, RProp Accuracy = 0.8573, Hybrid Accuracy = 0.8449
Run 7: Adam Accuracy = 0.7917, RProp Accuracy = 0.8582, Hybrid Accuracy = 0.8449
Run 8: Adam Accuracy = 0.7663, RProp Accuracy = 0.8506, Hybrid Accuracy = 0.8610
Run 9: Adam Accuracy = 0.7698, RProp Accuracy = 0.8533, Hybrid Accuracy = 0.8235
Run 10: Adam Accuracy = 0.7886, RProp Accuracy = 0.8595, Hybrid Accuracy = 0.8289

Best Run: 8 with Hybrid Accuracy = 0.8610

Summary of Results:
Adam - Average Accuracy: 0.7770, Std Dev: 0.0121
RProp - Average Accuracy: 0.8535, Std Dev: 0.0068
Hybrid - Average Accuracy: